In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

In [3]:
train_dataset = MNIST(True)
test_dataset = MNIST(False)

In [4]:
train_x, train_y = train_dataset.train_data.numpy(), train_dataset.train_labels
test_x, test_y = test_dataset.test_data.numpy(), test_dataset.test_labels
train_y2 = to_one_hot(train_y, 10).numpy()
test_y2 = to_one_hot(test_y, 10).numpy()
train_y = train_y.numpy()
test_y = test_y.numpy()

In [5]:
train_data = FromNumpy(train_x[:, None, :, :], [train_y2, train_y], batch_size=256)
test_data = FromNumpy(test_x[:, None, :, :], [test_y2, test_y], batch_size=256)

In [6]:
class Auxiliary(nn.Module):
    def __init__(self, f):
        super().__init__()
        self.f = f
        self.store = None
    
    def forward(self, x):
        # Store the result
        # Return the origin value
        self.store = self.f(x)
        return x 

In [7]:
model = builder([
    nn.Conv2d(1, 20, 5, 1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(20, 50, 5, 1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    Flatten(),
    nn.Linear(4*4*50, 500),
    nn.ReLU(),
    nn.Linear(500, 10),
    Auxiliary(nn.Sigmoid()),
    nn.Softmax(1)
])

In [8]:
runner = Runner(model, train_data, test_data, 'adam', ['bce', 'll'])

In [9]:
runner.fit(4)

  Epochs    train_loss    train_acc      val_loss      val_acc    
    1*       -2.4463        0.9140       -2.6578        0.9797    


    2        -2.6714        0.9847       -2.6829        0.9897    


    3        -2.6879        0.9901       -2.6742        0.9854    


    4        -2.6897        0.9906       -2.6906        0.9911    



In [ ]:
model = runner.model

In [ ]:
data = next(iter(train_data))
x = data[0][0]
x = x[None, :, :, :]
out1 = model(x.cuda())
out2 = get_aux_out(model)[0]
print(out1)
print(out2)
print(out1*out2)
